# Analzying object detection results

In [1]:
import pandas as pd
import os
import json

In [2]:
results_folder = os.path.join("data", "results")
result_files = sorted([f for f in os.listdir(results_folder) if f.endswith(".json")], reverse=True)
result_files

['r_flask_object-detection-XTRA-SMALL_2023-07-11T18:17:22.263557Z.json',
 'r_flask_object-detection-XTRA-SMALL_2023-07-11T17:15:33.487924Z.json',
 'r_flask_object-detection-SMALL_2023-07-11T19:35:53.673574Z.json',
 'r_flask_object-detection-SMALL_2023-07-11T17:15:44.414486Z.json',
 'r_flask_object-detection-MEDIUM_2023-07-11T19:37:05.239133Z.json',
 'r_flask_object-detection-MEDIUM_2023-07-11T17:17:45.200557Z.json',
 'r_flask_object-detection-BIG_2023-07-11T19:47:42.669375Z.json',
 'r_flask_object-detection-BIG_2023-07-11T17:26:00.861092Z.json',
 'l_tf-serving_object-detection-XTRA-SMALL(resnet50_v1_fpn_640x640)_2023-07-11T20:28:59.015023Z.json',
 'l_flask_object-detection-XTRA-SMALL_2023-07-11T17:13:18.977769Z.json',
 'l_flask_object-detection-SMALL_2023-07-11T19:53:02.663035Z.json',
 'l_flask_object-detection-SMALL_2023-07-11T15:32:51.393759Z.json',
 'l_flask_object-detection-MEDIUM_2023-07-11T19:50:58.413323Z.json',
 'l_flask_object-detection-MEDIUM_2023-07-11T15:33:48.661918Z.json'

In [3]:
result_file_paths = [os.path.join(results_folder, f) for f in result_files]

In [4]:
def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

## Flask API

In [5]:
flask_result_paths = [s for s in result_file_paths if "flask" in s]

Let's look at an example result for an 'extra small' dataset we created

In [6]:
example_result = read_json([p for p in flask_result_paths if 'XTRA-SMALL' in p][0])

In [7]:
list(example_result.keys())

['api_response',
 'data_transfer_time',
 'upload_time',
 'server_processing_time',
 'total_request_time',
 'request_sent_at',
 'input_folder_name',
 'api_url',
 'model']

What did the server return?

In [8]:
list(example_result['api_response'].keys())

['avg_inf_time',
 'inf_time',
 'predictions',
 'processing_time',
 'request_received_at']

In [9]:
example_result['api_response']['avg_inf_time']

'0.1585575739542643'

In [10]:
inf_times = pd.DataFrame(example_result['api_response']['inf_time'])
inf_times.describe()

,0
count,9.000000
mean,0.158558
std,0.004048
min,0.152420
25%,0.155797
50%,0.157853
75%,0.160777
max,0.165275


Looks like there's not too much variation in the inference speed.

We could spend a lot of time digging into the details of the results, but let's just get a general idea by collecting the most information from every combination of dataset (size), model, and execution environment (local or remote).

In [14]:
def extract_key_stats_series(result_file_path):
    result = read_json(result_file_path)
    api_response = result['api_response']
    avg_inf_time = api_response['avg_inf_time']
    environment = 'local' if 'localhost' in result['api_url'] else 'remote'
    model = result['model']
    upload_time = result['upload_time']
    total_request_time = result['total_request_time']
    server_processing_time = result['server_processing_time']
    dataset_variant = result['input_folder_name'].split('object-detection-')[1]
    request_date = result['request_sent_at']
    return pd.Series({
        'avg_inf_time': avg_inf_time,
        'total_request_time': total_request_time,
        'environment': environment,
        'model': model,
        'upload_time': upload_time,
        'total_request_time': total_request_time,
        'server_processing_time': server_processing_time,
        'dataset_variant': dataset_variant,
        'date': request_date
    })


In [15]:
extract_key_stats_series(flask_result_paths[0])

avg_inf_time                       0.1585575739542643
total_request_time                           2.174498
environment                                    remote
model                         resnet50_v1_fpn_640x640
upload_time                                   0.03073
server_processing_time                       2.091907
dataset_variant                            XTRA-SMALL
date                      2023-07-11T18:17:22.263557Z
dtype: object

In [112]:
flask_results = pd.concat([extract_key_stats_series(p) for p in flask_result_paths if 'XTRA' not in p], axis=1).T
flask_results

,avg_inf_time,total_request_time,environment,model,upload_time,server_processing_time,dataset_variant,date
0,0.03836536971298424,28.418084,remote,ssd_mobilenet_v2,0.068413,28.182802,SMALL,2023-07-11T19:35:53.673574Z
1,0.17710512795963804,69.477617,remote,resnet50_v1_fpn_640x640,0.070597,69.271074,SMALL,2023-07-11T17:15:44.414486Z
2,0.03731910134283241,199.409243,remote,ssd_mobilenet_v2,0.153222,198.325728,MEDIUM,2023-07-11T19:37:05.239133Z
3,0.17192468343725528,478.453536,remote,resnet50_v1_fpn_640x640,0.202784,477.239216,MEDIUM,2023-07-11T17:17:45.200557Z
4,0.03746230811420536,474.058114,remote,ssd_mobilenet_v2,0.360807,471.5447,BIG,2023-07-11T19:47:42.669375Z
5,0.15958795216523677,1075.328031,remote,resnet50_v1_fpn_640x640,0.370491,1072.843726,BIG,2023-07-11T17:26:00.861092Z
6,0.03470866905676352,10.626549,local,ssd_mobilenet_v2,0.02286,10.54293,SMALL,2023-07-11T19:53:02.663035Z
7,0.158267080783844,47.204244,local,resnet50_v1_fpn_640x640,0.021296,47.125437,SMALL,2023-07-11T15:32:51.393759Z
8,0.03289294047056189,70.892963,local,ssd_mobilenet_v2,0.140668,70.353443,MEDIUM,2023-07-11T19:50:58.413323Z
9,0.15617533478759912,325.725147,local,resnet50_v1_fpn_640x640,0.131383,325.203347,MEDIUM,2023-07-11T15:33:48.661918Z


Let's clean the data further

In [113]:
flask_results.date = pd.to_datetime(flask_results.date)
flask_results.model = flask_results.model.astype('category')
flask_results.model = flask_results.model.cat.rename_categories({'resnet50_v1_fpn_640x640': 'big', 'ssd_mobilenet_v2': 'small'})
flask_results.dataset_variant = flask_results.dataset_variant.astype('category')

# keep only most recent result for every model/dataset/environment combination
flask_results = flask_results.sort_values('date', ascending=False).drop_duplicates(['model', 'dataset_variant', 'environment'])
flask_results = flask_results.drop(columns=['date'])
flask_results = flask_results.sort_values(['model', 'dataset_variant', 'environment'], ascending=True)
flask_results = flask_results.reset_index(drop=True)
# cast numerical columns to float
flask_results[['avg_inf_time', 'upload_time', 'total_request_time', 'server_processing_time']] = flask_results[['avg_inf_time', 'upload_time', 'total_request_time', 'server_processing_time']].astype(float)

In [114]:
flask_results

,avg_inf_time,total_request_time,environment,model,upload_time,server_processing_time,dataset_variant
0,0.148923,738.089879,local,big,0.307885,736.774585,BIG
1,0.159588,1075.328031,remote,big,0.370491,1072.843726,BIG
2,0.156175,325.725147,local,big,0.131383,325.203347,MEDIUM
3,0.171925,478.453536,remote,big,0.202784,477.239216,MEDIUM
4,0.158267,47.204244,local,big,0.021296,47.125437,SMALL
5,0.177105,69.477617,remote,big,0.070597,69.271074,SMALL
6,0.032835,168.030120,local,small,0.328868,166.765619,BIG
7,0.037462,474.058114,remote,small,0.360807,471.544700,BIG
8,0.032893,70.892963,local,small,0.140668,70.353443,MEDIUM
9,0.037319,199.409243,remote,small,0.153222,198.325728,MEDIUM


In [115]:
flask_results.sort_values('total_request_time')

,avg_inf_time,total_request_time,environment,model,upload_time,server_processing_time,dataset_variant
10,0.034709,10.626549,local,small,0.022860,10.542930,SMALL
11,0.038365,28.418084,remote,small,0.068413,28.182802,SMALL
4,0.158267,47.204244,local,big,0.021296,47.125437,SMALL
5,0.177105,69.477617,remote,big,0.070597,69.271074,SMALL
8,0.032893,70.892963,local,small,0.140668,70.353443,MEDIUM
6,0.032835,168.030120,local,small,0.328868,166.765619,BIG
9,0.037319,199.409243,remote,small,0.153222,198.325728,MEDIUM
2,0.156175,325.725147,local,big,0.131383,325.203347,MEDIUM
7,0.037462,474.058114,remote,small,0.360807,471.544700,BIG
3,0.171925,478.453536,remote,big,0.202784,477.239216,MEDIUM


In [116]:
flask_results.groupby(['model', 'environment']).avg_inf_time.max() - flask_results.groupby(['model', 'environment']).avg_inf_time.min()

model  environment
big    local          0.009344
       remote         0.017517
small  local          0.001873
       remote         0.001046
Name: avg_inf_time, dtype: float64

The dataset size does not influence inference times at all.

In [117]:
flask_results

,avg_inf_time,total_request_time,environment,model,upload_time,server_processing_time,dataset_variant
0,0.148923,738.089879,local,big,0.307885,736.774585,BIG
1,0.159588,1075.328031,remote,big,0.370491,1072.843726,BIG
2,0.156175,325.725147,local,big,0.131383,325.203347,MEDIUM
3,0.171925,478.453536,remote,big,0.202784,477.239216,MEDIUM
4,0.158267,47.204244,local,big,0.021296,47.125437,SMALL
5,0.177105,69.477617,remote,big,0.070597,69.271074,SMALL
6,0.032835,168.030120,local,small,0.328868,166.765619,BIG
7,0.037462,474.058114,remote,small,0.360807,471.544700,BIG
8,0.032893,70.892963,local,small,0.140668,70.353443,MEDIUM
9,0.037319,199.409243,remote,small,0.153222,198.325728,MEDIUM


In [118]:
flask_results_local = flask_results[flask_results.environment == 'local'].drop(columns=['environment']).set_index(['model', 'dataset_variant']).sort_index()
flask_results_remote = flask_results[flask_results.environment == 'remote'].drop(columns=['environment']).set_index(['model', 'dataset_variant']).sort_index()

In [119]:
flask_results_local

avg_inf_time  total_request_time  upload_time  \
model dataset_variant                                                  
big   BIG                  0.148923          738.089879     0.307885   
      MEDIUM               0.156175          325.725147     0.131383   
      SMALL                0.158267           47.204244     0.021296   
small BIG                  0.032835          168.030120     0.328868   
      MEDIUM               0.032893           70.892963     0.140668   
      SMALL                0.034709           10.626549     0.022860   

                       server_processing_time  
model dataset_variant                          
big   BIG                          736.774585  
      MEDIUM                       325.203347  
      SMALL                         47.125437  
small BIG                          166.765619  
      MEDIUM                        70.353443  
      SMALL                         10.542930

In [120]:
flask_results_remote

avg_inf_time  total_request_time  upload_time  \
model dataset_variant                                                  
big   BIG                  0.159588         1075.328031     0.370491   
      MEDIUM               0.171925          478.453536     0.202784   
      SMALL                0.177105           69.477617     0.070597   
small BIG                  0.037462          474.058114     0.360807   
      MEDIUM               0.037319          199.409243     0.153222   
      SMALL                0.038365           28.418084     0.068413   

                       server_processing_time  
model dataset_variant                          
big   BIG                         1072.843726  
      MEDIUM                       477.239216  
      SMALL                         69.271074  
small BIG                          471.544700  
      MEDIUM                       198.325728  
      SMALL                         28.182802

In [128]:
def compute_perc_diffs_for_numeric_columns(df_a, df_b):
    return (df_a - df_b) / df_a

In [132]:
percentage_diffs_local_vs_remote = compute_perc_diffs_for_numeric_columns(flask_results_local, flask_results_remote)

In [133]:
percentage_diffs_local_vs_remote

avg_inf_time  total_request_time  upload_time  \
model dataset_variant                                                  
big   BIG                 -0.071615           -0.456907    -0.203342   
      MEDIUM              -0.100844           -0.468887    -0.543457   
      SMALL               -0.119027           -0.471851    -2.315036   
small BIG                 -0.140919           -1.821269    -0.097118   
      MEDIUM              -0.134563           -1.812821    -0.089246   
      SMALL               -0.105354           -1.674253    -1.992695   

                       server_processing_time  
model dataset_variant                          
big   BIG                           -0.456136  
      MEDIUM                        -0.467510  
      SMALL                         -0.469930  
small BIG                           -1.827589  
      MEDIUM                        -1.818991  
      SMALL                         -1.673147

In [137]:
def convert_df_perc_to_strs(df):
    return df.applymap(lambda x: f'{x:.2%}')

In [139]:
convert_df_perc_to_strs(percentage_diffs_local_vs_remote)

avg_inf_time total_request_time upload_time  \
model dataset_variant                                               
big   BIG                   -7.16%            -45.69%     -20.33%   
      MEDIUM               -10.08%            -46.89%     -54.35%   
      SMALL                -11.90%            -47.19%    -231.50%   
small BIG                  -14.09%           -182.13%      -9.71%   
      MEDIUM               -13.46%           -181.28%      -8.92%   
      SMALL                -10.54%           -167.43%    -199.27%   

                      server_processing_time  
model dataset_variant                         
big   BIG                            -45.61%  
      MEDIUM                         -46.75%  
      SMALL                          -46.99%  
small BIG                           -182.76%  
      MEDIUM                        -181.90%  
      SMALL                         -167.31%